# Prototype Transformer-Pipeline

In [1]:
import spacy
from spacy_layout import spaCyLayout
from pipeline_merge_pdf import process_pdf_folder
import os
import time

#Ordner, in dem PDFs liegen
pdf_folder = "../data/DSA/Der_blaue_Bruder"
#pdf_folder = "../data/DSA/Adel_Recht_und_Edelmut"
merged_pdf_path = os.path.join(pdf_folder, "_combined.pdf")

#Prüfen ob eine merged-Datei bereits vorhanden ist. Falls nicht, diese erzeugen
if os.path.exists(merged_pdf_path):
    print("Die Datei _combined.pdf existiert bereits unter folgendem Pfad:",merged_pdf_path)
    print("Lade die Datei lokal..\n")
    time.sleep(0.5)
    nlp = spacy.blank("de")
    layout = spaCyLayout(nlp)
    doc = layout(merged_pdf_path)
else:
    print("Die Datei _combined.pdf existiert noch nicht. Merge-Pipeline wird durchgeführt..")
    time.sleep(0.5)
    #Pipeline ausführen
    doc = process_pdf_folder(pdf_folder)

#spaCy-Doc
print("\nAnzahl Layout-Spans:", len(doc.spans["layout"]))


/home/timc/Workspace/nlp-rpg-narratives/nlp-rpg-venv/lib/python3.13/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/home/timc/Workspace/nlp-rpg-narratives/nlp-rpg-venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Die Datei _combined.pdf existiert bereits unter folgendem Pfad: ../data/DSA/Der_blaue_Bruder/_combined.pdf
Lade die Datei lokal..



2026-01-20 21:29:39,680 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-20 21:29:39,715 - INFO - Going to convert document batch...
2026-01-20 21:29:39,716 - INFO - Initializing pipeline for StandardPdfPipeline with options hash f9730ffaa6e7f8d4fb0c98c8df3f18cb
2026-01-20 21:29:39,727 - INFO - Loading plugin 'docling_defaults'
2026-01-20 21:29:39,729 - INFO - Registered picture descriptions: ['vlm', 'api']
2026-01-20 21:29:39,733 - INFO - Loading plugin 'docling_defaults'
2026-01-20 21:29:39,735 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2026-01-20 21:29:39,736 - INFO - rapidocr cannot be used because onnxruntime is not installed.
2026-01-20 21:29:39,736 - INFO - easyocr cannot be used because it is not installed.
2026-01-20 21:29:39,806 - INFO - Accelerator device: 'cuda:0'
[INFO] 2026-01-20 21:29:39,819 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-20 21:29:39,825 [RapidOCR] download_file.py:60: 


Anzahl Layout-Spans: 1833


In [2]:
from pipeline_preproc_pdf import process_doc_into_posts 

#Posts extrahieren + bereinigen
posts = process_doc_into_posts(doc)

#Posts ausgeben
for i, p in enumerate(posts):
    print(f"--- Post {i+1} ---\n{p}\n")

--- Post 1 ---
Die Grafschaft Heldentrutz zwischen Finsterkamm und Finsterbach ist eine der ärmsten und wildesten Regionen Weidens. Sie wird von zahlreichen dichten und gefährlichen Wäldern, mit Harpyien, Orks und Drachen, urwüchsigen Wiesen sowie von Heide- und Moorlandschaften geprägt. Neben ein wenig Kornwirtschaft betreiben die Leute vor allem Kohl- und Rübenanbau, sowie Viehwirtschaft, vor allem Schafe und Ziegen. Einzige  Ausnahme  in der ganzen Ärmlichkeit ist die Baronie Nordhag mit der gleichnamigen Stadt, die durch die dort vorhandenen Salzvorkommen zu Reichtum gekommen  ist.  Nordhag bildet das wirtschaftliche und kulturelle Zentrum der Grafschaft, liegt an wichtigen Handelswegen (Norrnstieg, Alter Weg,  Alte Straße), richtet zweimal  im Jahr den großen Wollmarkt aus und ist durch die Feste Norrbrug ganz gut geschützt. Am 10. Efferd des Jahres 1040 BF, zur Zeit des zweiten Wollmarktes leben in Nordhag um  die 900 Einwohner, man  findet dort Tempel  von Rondra, Phex, Peraine 

In [3]:
# Source - https://stackoverflow.com/a
# Posted by Ivan Procopovich, modified by community. See post 'Timeline' for change history
# Retrieved 2025-12-17, License - CC BY-SA 4.0

import sys
import time

def print_percent_done(index, total, bar_len=50, title='Please wait'):
    '''
    index is expected to be 0 based index. 
    0 <= index < total
    '''
    percent_done = (index+1)/total*100
    percent_done = round(percent_done, 1)

    done = round(percent_done/(100/bar_len))
    togo = bar_len-done

    done_str = '█'*int(done)
    togo_str = '░'*int(togo)

    print(f'\t⏳ {title}: [{done_str}{togo_str}] {percent_done}% done', end='\r')

    if round(percent_done) == 100:
        print('\t✅ ')


r = 50
for i in range(r):
    print_percent_done(i,r)
    time.sleep(.02)


	✅ Please wait: [██████████████████████████████████████████████████] 100.0% done


In [4]:
#Typische DSA-Wörter Pseudonymisieren, zur besseren Identifikation später
from rapidfuzz import fuzz
from nltk.tokenize import word_tokenize
from pathlib import Path

#print(posts[0])
#words = word_tokenize(posts[0], language="german")
#print(words)

for o, post in enumerate(posts): #Iterieren durch alle Posts
    #print("Verarbeite Post",o+1,"von",len(posts), end='\r')
    print_percent_done(o+1,len(posts))
    words = word_tokenize(post, language="german")
    for txt_file in Path("/home/timc/Workspace/nlp-rpg-narratives/").glob("*.txt"): #Iteriere durch alle Text-Files
        txt_file_tokens = str(txt_file).split("/")
        txt_name = txt_file_tokens[-1]
        txt_pseud = txt_name[:-4] #Nutzen den jeweiligen Namen des Text-Files als Pseudonym
        with open(txt_file, "r", encoding="utf-8") as f:
            names = [line.strip() for line in f if line.strip()]
            for name in names: #Iterieren durch alle Namen im jeweiligen Text-File
                for i, word in enumerate(words): #Iterieren durch alle Wörter aus dem jeweiligen Post
                    erg = fuzz.ratio(word, name)
                    if erg > 90.0:
                        words[i] = txt_pseud
                        #print(name,word,txt_pseud)
    posts[o] = ' '.join(words) #Ersetze alten Post in der Liste durch den neuen pseudonymisierten

#print("-----------------------------------------------------------")
#print(words)
#posts[0] = ' '.join(words)
print(posts[0])


	✅ Please wait: [██████████████████████████████████████████████████] 99.7% done
	✅ Please wait: [██████████████████████████████████████████████████] 100.0% done
	✅ Please wait: [██████████████████████████████████████████████████] 100.3% done
Die Grafschaft dsa_names zwischen dsa_names und dsa_names ist eine der ärmsten und wildesten Regionen dsa_names . Sie wird von zahlreichen dichten und gefährlichen Wäldern , mit Harpyien , Orks und dsa_names , urwüchsigen Wiesen sowie von Heide- und Moorlandschaften geprägt . Neben ein wenig Kornwirtschaft betreiben die Leute vor allem Kohl- und Rübenanbau , sowie Viehwirtschaft , vor allem Schafe und Ziegen . Einzige Ausnahme in der ganzen Ärmlichkeit ist die Baronie dsa_stadtnames mit der gleichnamigen Stadt , die durch die dort vorhandenen Salzvorkommen zu Reichtum gekommen ist . dsa_stadtnames bildet das wirtschaftliche und kulturelle Zentrum der Grafschaft , liegt an wichtigen Handelswegen ( Norrnstieg , Alter Weg , Alte Straße ) , richtet zwe

In [5]:
posts[2]

'dsa_names hat sich über die Einladung gefreut . Auch wenn sie sich nicht wirklich kennen , so doch zumindest vom Namen her und Verbindungen können nie schaden . Also lenkt sie ihr Pferd nach dsa_stadtnames oder besser auf die dsa_stadtnames . Entspannt reitet sie , gut gerüstet und bewaffnet und bestens gelaunt . Eine Reise ist immer nett und bisher sind die Verwaltungstätigkeiten ihres Lehens doch sehr überschaubar , also bleibt Zeit zum Reisen .'

In [8]:
from transformers import pipeline

labels = [
    "quest_reise",
    "kampf_schlacht",
    "entdeckung_geheimnis",
    "verrat_intrige",
    "prophezeiung_schicksal",
    "rettung_opfer",
    "flucht_verfolgung",
    "herrschaft_macht"
]

classifier = pipeline("zero-shot-classification", model="./fantasy_model")
test = classifier(posts[2], candidate_labels=labels)
print(test['labels'])
print(test['scores'])

Device set to use cuda:0
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


['kampf_schlacht', 'verrat_intrige', 'rettung_opfer', 'quest_reise', 'flucht_verfolgung', 'herrschaft_macht', 'prophezeiung_schicksal', 'entdeckung_geheimnis']
[0.1263367235660553, 0.12602795660495758, 0.12542320787906647, 0.12484876066446304, 0.12482740730047226, 0.12474830448627472, 0.1239914670586586, 0.12379614263772964]


In [10]:
print(test['labels'][0])
print(test['scores'][0])

kampf_schlacht
0.1263367235660553


In [33]:
for beitrag in posts:
    erg = classifier(beitrag, candidate_labels=labels)
    print("Label",erg['labels'][0],"---->",erg['scores'][0])
    #print("-------")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Label verrat_intrige ----> 0.1254778355360031
Label rettung_opfer ----> 0.12662522494792938
Label kampf_schlacht ----> 0.1263367235660553
Label verrat_intrige ----> 0.12688012421131134
Label verrat_intrige ----> 0.12642501294612885
Label verrat_intrige ----> 0.12567178905010223
Label verrat_intrige ----> 0.12637437880039215
Label prophezeiung_schicksal ----> 0.1278965324163437
Label verrat_intrige ----> 0.1258998066186905
Label verrat_intrige ----> 0.12675265967845917
Label verrat_intrige ----> 0.12649750709533691
Label rettung_opfer ----> 0.1264055222272873
Label entdeckung_geheimnis ----> 0.12593775987625122
Label rettung_opfer ----> 0.12579335272312164
Label verrat_intrige ----> 0.12657113373279572
Label verrat_intrige ----> 0.12657277286052704
Label rettung_opfer ----> 0.1272980272769928
Label rettung_opfer ----> 0.12599411606788635
Label rettung_opfer ----> 0.12674535810947418
Label verrat_intrige ----> 0.12578970193862915
Label kampf_schlacht ----> 0.12570562958717346
Label verra